17.1

Zadaniem jest stworzenie prostego interfejsu programistycznego (API) przy użyciu frameworku Flask w języku Python.

Twoje API powinno obsługiwać zapytania HTTP GET i POST, umoźliwiając uźytkownikowi pobieranie informacji oraz dodawanie nowych danych.

In [1]:
pip install flask

In [2]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# Przykładowe dane
data = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob", "age": 30}
]

# Endpoint GET: Pobieranie danych
@app.route('/api/data', methods=['GET'])
def get_data():
    return jsonify(data)

# Endpoint POST: Dodawanie nowych danych
@app.route('/api/data', methods=['POST'])
def add_data():
    if request.is_json:
        new_entry = request.get_json()
        new_entry["id"] = len(data) + 1  # Automatyczne ustawianie ID
        data.append(new_entry)
        return jsonify({"message": "Data added successfully", "data": new_entry}), 201
    return jsonify({"message": "Request must be JSON"}), 400

# Uruchamianie serwera
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
curl http://127.0.0.1:5000/api/data

In [ ]:
curl -X POST http://127.0.0.1:5000/api/data \
     -H "Content-Type: application/json" \
     -d '{"name": "Charlie", "age": 22}'

17.2

Zadaniem jest stworzenie pliku docker-compose.yml do konteneryzacji dwóch usług: aplikacji Pythona opartej na Flask oraz bazy danych Redis. Aplikacja Pythona powinna korzystać z bazy danych Redis do przechowywania prostych wiadomości.

In [ ]:
.
âââ docker-compose.yml
âââ Dockerfile
âââ app
    âââ app.py
    âââ requirements.txt

In [ ]:
version: '3.8'

services:
  flask-app:
    build: .
    ports:
      - "5000:5000"
    environment:
      - REDIS_HOST=redis
    depends_on:
      - redis
    networks:
      - app-network

  redis:
    image: redis:latest
    ports:
      - "6379:6379"
    networks:
      - app-network

networks:
  app-network:
    driver: bridge

In [ ]:
# Uźycie oficjalnego obrazu Pythona
FROM python:3.10-slim

# Ustawienie katalogu roboczego
WORKDIR /app

# Kopiowanie plików aplikacji
COPY app/ /app/

# Instalacja zależności
RUN pip install --no-cache-dir -r requirements.txt

# Uruchomienie aplikacji
CMD ["python", "app.py"]

In [ ]:
Flask==2.3.2
redis==5.0.0

In [ ]:
from flask import Flask, request, jsonify
import redis
import os

app = Flask(__name__)

# Konfiguracja połączenia z Redis
redis_host = os.getenv("REDIS_HOST", "localhost")
redis_client = redis.StrictRedis(host=redis_host, port=6379, decode_responses=True)

# Endpoint: Dodanie wiadomości
@app.route('/add_message', methods=['POST'])
def add_message():
    data = request.get_json()
    if not data or 'message' not in data:
        return jsonify({"error": "Message is required"}), 400
    message_id = redis_client.incr("message_id")
    redis_client.set(f"message:{message_id}", data['message'])
    return jsonify({"message": "Message added", "id": message_id}), 201

# Endpoint: Pobranie wiadomości
@app.route('/get_message/<int:message_id>', methods=['GET'])
def get_message(message_id):
    message = redis_client.get(f"message:{message_id}")
    if message:
        return jsonify({"id": message_id, "message": message}), 200
    return jsonify({"error": "Message not found"}), 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
docker-compose up --build

In [ ]:
curl -X POST http://127.0.0.1:5000/add_message \
     -H "Content-Type: application/json" \
     -d '{"message": "Hello, Redis!"}'

curl http://127.0.0.1:5000/get_message/1

17.3

Zadaniem jest wdroźenie modelu mlflow:
- skorzystaj z wybranego zbioru danych biblioteki np. sklearn,
- przygotuj dane,
- zaimplementuj wybrany model,
- uźyj mlflow do zapisania modelu po treningu,
- zapisz model mlflow i zweryfikuj poprawność modelu na zbiorze testowym.

In [3]:
pip install mlflow

   ââââââââââââââââââââââââââââââââââââââââ 27.3/27.3 MB 54.1 MB/s eta 0:00:00
   ââââââââââââââââââââââââââââââââââââââââ 5.8/5.8 MB 79.5 MB/s eta 0:00:00
   ââââââââââââââââââââââââââââââââââââââââ 233.5/233.5 kB 18.0 MB/s eta 0:00:00
   ââââââââââââââââââââââââââââââââââââââââ 147.8/147.8 kB 11.7 MB/s eta 0:00:00
   ââââââââââââââââââââââââââââââââââââââââ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ââââââââââââââââââââââââââââââââââââââââ 85.0/85.0 kB 6.3 MB/s eta 0:00:00
   ââââââââââââââ

In [5]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Wczytanie zbioru danych
iris = load_iris()
X = iris.data
y = iris.target

# Podział danych na zbiory treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Modelowanie: Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 3. Inicjalizacja MLflow
mlflow.set_experiment("iris_classification_experiment")

with mlflow.start_run() as run:
    # Trening modelu
    model.fit(X_train, y_train)

    # Predykcja
    y_pred = model.predict(X_test)

    # Obliczanie dokładności
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")

    # Logowanie parametrów
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)

    # Logowanie metryk
    mlflow.log_metric("accuracy", accuracy)

    # Logowanie modelu
    mlflow.sklearn.log_model(model, "random_forest_model")
    print("Model został zapisany w MLflow.")

    # Zapisywanie run_id
    run_id = run.info.run_id

# 4. Ładowanie modelu z MLflow i weryfikacja
logged_model = f"runs:/{run_id}/random_forest_model"

# Ładowanie modelu
loaded_model = mlflow.sklearn.load_model(logged_model)

# Weryfikacja na zbiorze testowym
y_loaded_pred = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, y_loaded_pred)
print(f"Accuracy załadowanego modelu: {loaded_accuracy}")

Accuracy: 1.0


2024/12/11 09:14:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model zostaĹ zapisany w MLflow.
Accuracy zaĹadowanego modelu: 1.0


In [ ]:
mlflow ui